<a href="https://colab.research.google.com/github/mosheber/try_auto_uav_architecture/blob/master/mtrl_auto_uav_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!wget https://zenodo.org/record/3270774/files/manual_4.zip

--2020-09-12 21:22:15--  https://zenodo.org/record/3270774/files/manual_4.zip
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5569974696 (5.2G) [application/octet-stream]
Saving to: ‘manual_4.zip’

manual_4.zip        100%[===================>]   5.19G  4.92MB/s    in 11m 29s 

2020-09-12 21:33:47 (7.71 MB/s) - ‘manual_4.zip’ saved [5569974696/5569974696]



In [8]:
!unzip manual_4.zip

Archive:  manual_4.zip
warning [manual_4.zip]:  1273994048 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [manual_4.zip]:  start of central directory not found;
  zipfile corrupt.
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [9]:
import torch

In [10]:
def get_cnn_output_size(N,k,F,s):
  return int((N+2*k-F)/s + 1)

In [36]:
from torch import nn
class ConvNet(nn.Module):  
  def __init__(self,image_size,
               input_channels=3,
               init_transform_channel_power=4,
               conv_layer_count=2,
               conv_kernel_size=3,
               conv_stride=1,
               max_pool_kernel_size=2,
               max_pool_stride=2,
               fc_layer_count=4,
               fc_dropout=0.3,
               fc_ned_dropout=0.5,
               fc_quat_dropout=0.25):
    super().__init__()
    self.image_size = image_size
    self.input_channels = input_channels
    self.init_transform_channel_power = init_transform_channel_power

    self.conv_layer_count=conv_layer_count
    self.conv_kernel_size=conv_kernel_size
    self.conv_stride=conv_stride
    self.max_pool_kernel_size=max_pool_kernel_size
    self.max_pool_stride=max_pool_stride
    self.fc_layer_count=fc_layer_count
    self.fc_dropout = fc_dropout
    self.fc_ned_dropout = fc_ned_dropout
    self.fc_quat_dropout = fc_quat_dropout

    self.shared_model = nn.Sequential(*self.get_conv_block(self.input_channels,int(2**self.init_transform_channel_power)))
    self.image_size = self.calc_size_after_block(self.image_size)

    base_layers_1,output_1 = self.get_base_model()
    ned_layers = self.get_ned_model(base_layers_1,output_1)
    self.ned_model = nn.Sequential(*ned_layers)

    base_layers_2,output_2 = self.get_base_model()
    quat_layers = self.get_quat_model(base_layers_2,output_2)
    self.quat_model = nn.Sequential(*quat_layers)


  def forward(self,X):
    output = self.shared_model(X)
    return self.ned_model(output), self.quat_model(output)

  def get_conv_block(self,input_channel,output_channel):
    return [
        nn.Conv2d(input_channel,output_channel,kernel_size=self.conv_kernel_size,stride=self.conv_stride),
        nn.MaxPool2d(kernel_size=self.max_pool_kernel_size,stride=self.max_pool_stride),
        nn.BatchNorm2d(output_channel)
    ]
    
  def calc_size_after_block(self,N):
    N = get_cnn_output_size(N,0,self.conv_kernel_size,self.conv_stride)
    N = get_cnn_output_size(N,0,self.max_pool_kernel_size,self.max_pool_stride)
    return N

  def get_conv_layers(self):
    layers = []
    N = self.image_size
    for i in range(self.conv_layer_count):      
      input_channel = int(2**(i+self.init_transform_channel_power))
      output_channel = int(2**(i+self.init_transform_channel_power+1))
      N = self.calc_size_after_block(N)
      layers = layers + self.get_conv_block(input_channel,output_channel)
      
    print(f'layers: {layers}, N: {N}, output_channel: {output_channel}')
    return layers, N, output_channel

  def get_fc_layers(self,image_size, output_channel):
    flattened_size = (image_size**2) * output_channel
    print(f'flattened_size: {flattened_size}')
    layers = []
    for i in range(self.fc_layer_count):
      input_size = int(500/(5**(i-1)))
      output_size = int(500/(5**(i)))
      if(i==0):
        input_size = flattened_size

      layers = layers + [
        nn.Linear(input_size,output_size),
        nn.Dropout(self.fc_dropout),
      ]

    print(f'layers: {layers}, output_size: {output_size}')
    return layers,output_size

  def get_base_model(self):
    conv_layers, N, output_channel = self.get_conv_layers()
    fc_layers, output_size = self.get_fc_layers(N, output_channel)
    return conv_layers + [nn.Flatten()] + fc_layers, output_size

  def get_ned_model(self,layers,output_size):
    return layers + [nn.Linear(output_size,3)]
  
  def get_quat_model(self,layers,output_size):
    return layers + [nn.Linear(output_size,4)]


In [37]:
image_size = 224
channel_size = 3
image_count = 20

input = torch.randn(image_count, channel_size, image_size, image_size)

In [38]:
model = ConvNet(image_size)

layers: [Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1)), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1)), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)], N: 26, output_channel: 64
flattened_size: 43264
layers: [Linear(in_features=43264, out_features=500, bias=True), Dropout(p=0.3, inplace=False), Linear(in_features=500, out_features=100, bias=True), Dropout(p=0.3, inplace=False), Linear(in_features=100, out_features=20, bias=True), Dropout(p=0.3, inplace=False), Linear(in_features=20, out_features=4, bias=True), Dropout(p=0.3, inplace=False)], output_size: 4
layers: [Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1)), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), BatchNorm2d(32, eps=1e-05, 

In [39]:
ned,quat = model(input)

In [41]:
ned.shape

torch.Size([20, 3])

In [42]:
quat.shape

torch.Size([20, 4])